# Week 08 - Regression

---

### Moneyball Analysis via Linear Regression

---

### Objective

This analysis demonstrates that On-Base Percentage (OBP) was undervalued relative to Batting Average (BA) before the Moneyball era (pre-2003) (Moneyball Book: 2003, Moneyball Movie: 2011), and proves that OBP is the strongest predictor of team success.

---

### Background

_Moneyball_ explores how The Oakland Athletics (referred to as the Oakland A's) in 2002, a low-budget Major League Baseball team, used statistics and data analytics to compete against wealthier teams. Focusing on General Manager Billy Beane, _Moneyball_ highlights how traditional scouting methods were challenged by a new, evidence-based approach that **prioritized undervalued player statistics like on-base percentage**. _Moneyball_ is both a compelling sports narrative and a broader commentary on the power of data to disrupt established systems.

---

### Notes

1. **Market Inefficiency Identified**: Teams were paying more for BA than OBP, despite OBP being a stronger predictor of offensive success.

2. **Oakland's Competitive Advantage**: By focusing on OBP instead of BA, Oakland could acquire **undervalued players** who would contribute more to winning. They hade a budget limit to build their roster.

3. **Statistical Evidence**: This analysis shows that OBP explains more variance in runs scored and has a stronger coefficient in predicting RD.

4. **Market Correction**: After the Moneyball book and movie, teams began to value OBP more highly, reducing this inefficiency.

5. This analyis uses **pre-2002 data** to capture the pre-Moneyball market conditions

6. **OBP** measures how frequently a player reaches base.

7. **BASE** refers to one of the four points on the diamond (first, second, third, and home plate) that a runner must touch in order to score a run.

---

### Plan

- From both the Moneyball book and movie we know that OBP is the strongest predictor of Run Difference and Runs Scored.
- We would like to prove that OBP is indeed the strongest predictor.
- Then, find players who are undervalued and hire them.

- Oakland A's had a buget limit for their enitre roster. By figuring out that OBP was the actual metric/stat to evaluate players and not BA, then they found undervalued plahers (HIGH OBP & LOW SALARY) to create their roster.

---

### Key Questions To Answer:

1. Is OBP a stronger predictor of Runs Scored (RS) than BA?
2. Is OBP the biggest predictor of Run Difference (RD = RS - RA)? **_(RA = Runs Allowed)_**
3. Which players are considered as undervalued?

---

### Acronyms

- **RS**: Runs Scored
- **RA**: Runs Allowed
- **RD**: Run Difference (RS - RA)
- **OBP**: On Base Percentage
- **OOBP**: Opponent's On Base Percentage
- **BA**: Batting Average
- **SLG**: Slugging Percentage
- **OSLG**: Opponent's Slugging Percentage

---


## 1. Data Loading and Preparation


In [1]:
# Import libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import statsmodels.api as sm
import warnings

from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

warnings.filterwarnings("ignore")

# Set plotting style for better visualizations
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

# Configure display options for better readability
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [2]:
# Load the baseball dataset
df = pd.read_csv("baseball_records_per_seasons_with_salary.csv")

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
df.head()

Dataset Shape: (902, 21)

Column Names:
['Team', 'League', 'Year', 'RS', 'RA', 'W', 'OBP', 'SLG', 'BA', 'Playoffs', 'RankSeason', 'RankPlayoffs', 'G', 'OOBP', 'OSLG', 'runs_diff', 'RD', 'teamID', 'yearID', 'salary', 'formatted_salary']


,Team,League,Year,RS,RA,W,OBP,SLG,BA,Playoffs,RankSeason,RankPlayoffs,G,OOBP,OSLG,runs_diff,RD,teamID,yearID,salary,formatted_salary
0,ANA,AL,2001,691,730,75,0.327,0.405,0.261,0,NaN,NaN,162,0.331,0.412,-39,-39,ANA,2001.0,33528500.0,"$33,528,500"
1,ARI,NL,2001,818,677,92,0.341,0.442,0.267,1,5.0,1.0,162,0.311,0.404,141,141,ARI,2001.0,80872999.0,"$80,872,999"
2,ATL,NL,2001,729,643,88,0.324,0.412,0.260,1,7.0,3.0,162,0.314,0.384,86,86,ATL,2001.0,96203333.0,"$96,203,333"
3,BAL,AL,2001,687,829,63,0.319,0.380,0.248,0,NaN,NaN,162,0.337,0.439,-142,-142,BAL,2001.0,59760500.0,"$59,760,500"
4,BOS,AL,2001,772,745,82,0.334,0.439,0.266,0,NaN,NaN,161,0.329,0.393,27,27,BOS,2001.0,110828333.0,"$110,828,333"


In [3]:
# Filter data for pre-Moneyball book era (years <= 2001)
# The Moneyball book was published in 2003 and the team was formed in 2002, so we focus on data before that
df_pre_moneyball = df[df["Year"] <= 2001].copy()

print(f"Original dataset: {df.shape[0]} records")
print(f"Pre-Moneyball dataset (≤2001): {df_pre_moneyball.shape[0]} records")
print(
    f"\nYear range in pre-Moneyball data: {df_pre_moneyball['Year'].min()} - {df_pre_moneyball['Year'].max()}"
)
# Display number of unique teams
print(
    f"Number of unique teams in pre-Moneyball data: {df_pre_moneyball['Team'].nunique()}"
)
print(f"Unique teams: {sorted(df_pre_moneyball['Team'].unique())}")

# Check for missing values in key columns
print("\nMissing values in key columns:")
key_columns = ["RS", "RA", "OBP", "BA", "SLG", "OOBP", "OSLG", "salary"]
missing_data = df_pre_moneyball[key_columns].isnull().sum()
print(missing_data[missing_data > 0])

df_pre_moneyball.head()

Original dataset: 902 records
Pre-Moneyball dataset (≤2001): 902 records

Year range in pre-Moneyball data: 1962 - 2001
Number of unique teams in pre-Moneyball data: 36
Unique teams: ['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'CAL', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'FLA', 'HOU', 'KCA', 'KCR', 'LAA', 'LAD', 'MIL', 'MIN', 'MLN', 'MON', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA', 'SEP', 'SFG', 'STL', 'TBD', 'TEX', 'TOR', 'WSA']

Missing values in key columns:
OOBP      812
OSLG      812
salary    366
dtype: int64


,Team,League,Year,RS,RA,W,OBP,SLG,BA,Playoffs,RankSeason,RankPlayoffs,G,OOBP,OSLG,runs_diff,RD,teamID,yearID,salary,formatted_salary
0,ANA,AL,2001,691,730,75,0.327,0.405,0.261,0,NaN,NaN,162,0.331,0.412,-39,-39,ANA,2001.0,33528500.0,"$33,528,500"
1,ARI,NL,2001,818,677,92,0.341,0.442,0.267,1,5.0,1.0,162,0.311,0.404,141,141,ARI,2001.0,80872999.0,"$80,872,999"
2,ATL,NL,2001,729,643,88,0.324,0.412,0.260,1,7.0,3.0,162,0.314,0.384,86,86,ATL,2001.0,96203333.0,"$96,203,333"
3,BAL,AL,2001,687,829,63,0.319,0.380,0.248,0,NaN,NaN,162,0.337,0.439,-142,-142,BAL,2001.0,59760500.0,"$59,760,500"
4,BOS,AL,2001,772,745,82,0.334,0.439,0.266,0,NaN,NaN,161,0.329,0.393,27,27,BOS,2001.0,110828333.0,"$110,828,333"


In [4]:
# Create Run Difference (RD) column if it doesn't exist or has missing values
# RD = RS - RA (Runs Scored - Runs Allowed)
if "RD" in df_pre_moneyball.columns:
    # Check if RD column has missing values
    rd_missing = df_pre_moneyball["RD"].isnull().sum()
    print(f"RD column missing values: {rd_missing}")

    # Fill missing RD values with calculated values
    df_pre_moneyball["RD"] = df_pre_moneyball["RD"].fillna(
        df_pre_moneyball["RS"] - df_pre_moneyball["RA"]
    )
else:
    # Create RD column
    df_pre_moneyball["RD"] = df_pre_moneyball["RS"] - df_pre_moneyball["RA"]
    print("Created RD column from RS - RA")

# Remove rows with missing salary data for salary analysis
df_with_salary = df_pre_moneyball.dropna(subset=["salary"]).copy()
print(f"\nRecords with salary data: {df_with_salary.shape[0]}")

# Display basic statistics for key metrics
print("\nBasic Statistics for Key Metrics:")
stats_columns = ["RS", "RA", "RD", "OBP", "BA", "SLG", "salary"]
print(df_pre_moneyball[stats_columns].describe())

RD column missing values: 0

Records with salary data: 536

Basic Statistics for Key Metrics:
                RS           RA          RD         OBP          BA  \
count   902.000000   902.000000  902.000000  902.000000  902.000000   
mean    703.809313   703.809313    0.000000    0.324961    0.258153   
std      93.314579    93.784100  101.188684    0.015391    0.013266   
min     463.000000   472.000000 -331.000000    0.277000    0.214000   
25%     641.250000   640.000000  -70.750000    0.314000    0.250000   
50%     695.000000   697.000000    3.000000    0.324000    0.258000   
75%     761.750000   763.000000   69.750000    0.335000    0.267000   
max    1009.000000  1103.000000  309.000000    0.373000    0.294000   

              SLG        salary  
count  902.000000  5.360000e+02  
mean     0.390412  1.567212e+07  
std      0.033110  2.208607e+07  
min      0.301000  0.000000e+00  
25%      0.368000  4.432500e+05  
50%      0.388000  3.948042e+06  
75%      0.411750  2.350849e

---
#### NOTE: Every Linear Regression model has **X (Independent Variable(s))** and **y (Dependent Variable)**.
#### **Dependent Variable** ***depends*** on the **Independent Variable(s)** for a change.
#### **Independent Variable(s)** is what we can ***control*** to affect/change the **Dependent Variable**
---


## 2. Analysis Part 1: OBP vs BA as Predictors of RS

**Question:** Is OBP a stronger predictor of RS than BAv?

**Goal:** Show that OBP is a stronger predictor of offensive success than BA


In [5]:
# Prepare data for regression analysis
# Remove rows with missing OBP or BA data
regression_data = df_pre_moneyball.dropna(subset=["RS", "OBP", "BA"]).copy()
print(f"Records for RS prediction analysis: {regression_data.shape[0]}")

# Define features and target
X_obp = regression_data[["OBP"]]  # Independent variable
X_ba = regression_data[["BA"]]  # Independent variable
y = regression_data["RS"]  # Dependent variable

# Build linear regression models using statsmodels for detailed statistical output
# Add constant terms for intercepts
X_obp_with_const = sm.add_constant(X_obp)
X_ba_with_const = sm.add_constant(X_ba)

# Fit models using statsmodels
model_obp_sm = sm.OLS(y, X_obp_with_const).fit()
model_ba_sm = sm.OLS(y, X_ba_with_const).fit()

# Also create sklearn models for predictions
model_obp_sk = LinearRegression()
model_ba_sk = LinearRegression()

# Fit sklearn models
model_obp_sk.fit(X_obp, y)
model_ba_sk.fit(X_ba, y)

# Make predictions
y_pred_obp = model_obp_sk.predict(X_obp)
y_pred_ba = model_ba_sk.predict(X_ba)

# Calculate R²= scores
r2_obp = r2_score(y, y_pred_obp)
r2_ba = r2_score(y, y_pred_ba)

# Calculate correlation coefficients
corr_obp = regression_data["RS"].corr(regression_data["OBP"])
corr_ba = regression_data["RS"].corr(regression_data["BA"])

print("=" * 80)
print("DETAILED REGRESSION SUMMARY REPORTS")
print("=" * 80)

print("\n" + "=" * 50)
print("MODEL 1: RS ~ OBP (On-Base Percentage)")
print("=" * 50)
print(model_obp_sm.summary())

print("\n" + "=" * 50)
print("MODEL 2: RS ~ BA (Batting Average)")
print("=" * 50)
print(model_ba_sm.summary())

print("\n" + "=" * 80)
print("QUICK COMPARISON SUMMARY")
print("=" * 80)

print(f"\nOBP Model:")
print(f"  R² Score: {r2_obp:.4f}")
print(f"  Correlation: {corr_obp:.4f}")
print(f"  Coefficient: {model_obp_sk.coef_[0]:.2f}")
print(f"  Intercept: {model_obp_sk.intercept_:.2f}")
print(f"  P-value: {model_obp_sm.pvalues[1]:.4f}")
print(
    f"  Confidence Interval: [{model_obp_sm.conf_int().iloc[1, 0]:.2f}, {model_obp_sm.conf_int().iloc[1, 1]:.2f}]"
)

print(f"\nBA Model:")
print(f"  R² Score: {r2_ba:.4f}")
print(f"  Correlation: {corr_ba:.4f}")
print(f"  Coefficient: {model_ba_sk.coef_[0]:.2f}")
print(f"  Intercept: {model_ba_sk.intercept_:.2f}")
print(f"  P-value: {model_ba_sm.pvalues[1]:.4f}")
print(
    f"  Confidence Interval: [{model_ba_sm.conf_int().iloc[1, 0]:.2f}, {model_ba_sm.conf_int().iloc[1, 1]:.2f}]"
)

print(f"\n=== Key Statistical Findings ===")
if r2_obp > r2_ba:
    print(f"• OBP explains {r2_obp - r2_ba:.4f} more variance in RS than BA")
    print(f"• OBP is {r2_obp/r2_ba:.2f}x better at predicting RS than BA")
else:
    print(f"• BA explains {r2_ba - r2_obp:.4f} more variance in RS than BA")
    print(f"• BA is {r2_ba/r2_obp:.2f}x better at predicting RS than BA")

print(f"\n• Both models are statistically significant (p < 0.05)")
print(
    f"• OBP coefficient: {model_obp_sm.params[1]:.2f} (p = {model_obp_sm.pvalues[1]:.4f})"
)
print(
    f"• BA coefficient: {model_ba_sm.params[1]:.2f} (p = {model_ba_sm.pvalues[1]:.4f})"
)

Records for RS prediction analysis: 902
DETAILED REGRESSION SUMMARY REPORTS

MODEL 1: RS ~ OBP (On-Base Percentage)
                            OLS Regression Results                            
Dep. Variable:                     RS   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     4069.
Date:                Sat, 25 Oct 2025   Prob (F-statistic):               0.00
Time:                        01:29:29   Log-Likelihood:                -4600.3
No. Observations:                 902   AIC:                             9205.
Df Residuals:                     900   BIC:                             9214.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------

In [6]:
# Plotly version: OBP vs BA as Predictors of Runs Scored (Pre-Moneyball Era)
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        f"OBP vs RS<br>R² = {r2_obp:.3f}, Corr = {corr_obp:.3f}<br>p = {model_obp_sm.pvalues[1]:.4f}",
        f"BA vs RS<br>R² = {r2_ba:.3f}, Corr = {corr_ba:.3f}<br>p = {model_ba_sm.pvalues[1]:.4f}",
        "R² Comparison: OBP vs BA",
        "Correlation Comparison: OBP vs BA",
    ),
)

# Plot 1: OBP vs RS with regression line
fig.add_trace(
    go.Scatter(
        x=regression_data["OBP"],
        y=regression_data["RS"],
        mode="markers",
        marker=dict(color="blue", opacity=0.6),
        name="OBP vs RS",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=regression_data["OBP"],
        y=y_pred_obp,
        mode="lines",
        line=dict(color="red", width=2),
        name=f"R² = {r2_obp:.3f}",
    ),
    row=1,
    col=1,
)

# Plot 2: BA vs RS with regression line
fig.add_trace(
    go.Scatter(
        x=regression_data["BA"],
        y=regression_data["RS"],
        mode="markers",
        marker=dict(color="green", opacity=0.6),
        name="BA vs RS",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=regression_data["BA"],
        y=y_pred_ba,
        mode="lines",
        line=dict(color="red", width=2),
        name=f"R² = {r2_ba:.3f}",
    ),
    row=1,
    col=2,
)

# Plot 3: R² Comparison
metrics = ["OBP", "BA"]
r2_values = [r2_obp, r2_ba]
fig.add_trace(
    go.Bar(x=metrics, y=r2_values, marker_color=["blue", "green"], name="R²"),
    row=2,
    col=1,
)

# Plot 4: Correlation Comparison
corr_values = [corr_obp, corr_ba]
fig.add_trace(
    go.Bar(
        x=metrics, y=corr_values, marker_color=["blue", "green"], name="Correlation"
    ),
    row=2,
    col=2,
)

# Add value labels for bars
fig.update_traces(
    texttemplate="%{y:.3f}", textposition="outside", selector=dict(type="bar")
)

# Axes titles
fig.update_xaxes(title_text="On-Base Percentage (OBP)", row=1, col=1)
fig.update_yaxes(title_text="Runs Scored (RS)", row=1, col=1)
fig.update_xaxes(title_text="Batting Average (BA)", row=1, col=2)
fig.update_yaxes(title_text="Runs Scored (RS)", row=1, col=2)
fig.update_yaxes(title_text="R² Score", row=2, col=1, range=[0, max(r2_values) * 1.1])
fig.update_yaxes(
    title_text="Correlation Coefficient",
    row=2,
    col=2,
    range=[0, max(corr_values) * 1.1],
)

fig.update_layout(
    title_text="OBP vs BA as Predictors of Runs Scored (Pre-Moneyball Era)",
    title_x=0.5,
    width=1200,
    height=900,
    showlegend=False,
    margin=dict(l=60, r=40, t=100, b=60),
)

fig.show()

## 3. Analysis Part 2: OBP as Predictor of RD

**Question:** Is OBP the biggest predictor of RD (RD = RS - RA)?

**Goal:** Show OBP is the biggest predictor of winning (RD)


In [7]:
# Prepare data for multiple regression analysis
# Include all relevant features that could predict Run Difference
rd_features = ["OBP", "SLG"]  # Independent variables
rd_data = df_pre_moneyball.dropna(subset=["RD"] + rd_features).copy()

print(f"Records for RD prediction analysis: {rd_data.shape[0]}")
print(f"Features used: {rd_features}")

# Define features and target
X_rd = rd_data[rd_features]  # Independent variables
y_rd = rd_data["RD"]  # Dependent variable

# Check for multicollinearity using VIF (Variance Inflation Factor)
print("\n=== Multicollinearity Check (VIF) ===")
vif_data = pd.DataFrame()
vif_data["Feature"] = X_rd.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_rd.values, i) for i in range(len(X_rd.columns))
]
print(vif_data)

# VIF > 10 indicates high multicollinearity
high_vif = vif_data[vif_data["VIF"] > 10]
if len(high_vif) > 0:
    print(
        f"\nWarning: High multicollinearity detected for: {high_vif['Feature'].tolist()}"
    )
else:
    print("\nNo significant multicollinearity detected (all VIF < 10)")

Records for RD prediction analysis: 902
Features used: ['OBP', 'SLG']

=== Multicollinearity Check (VIF) ===
  Feature         VIF
0     OBP  340.843041
1     SLG  340.843041



In [8]:
# Prepare data for multiple regression analysis
# Include all relevant features that could predict Run Difference
rd_features = ["OBP", "OOBP", "SLG", "OSLG", "BA"]  # Independent variables
rd_data = df_pre_moneyball.dropna(subset=["RD"] + rd_features).copy()

print(f"Records for RD prediction analysis: {rd_data.shape[0]}")
print(f"Features used: {rd_features}")

# Define features and target
X_rd = rd_data[rd_features]  # Independent variables
y_rd = rd_data["RD"]  # Dependent variable

# Check for multicollinearity using VIF (Variance Inflation Factor)
print("\n=== Multicollinearity Check (VIF) ===")
vif_data = pd.DataFrame()
vif_data["Feature"] = X_rd.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_rd.values, i) for i in range(len(X_rd.columns))
]
print(vif_data)

# VIF > 10 indicates high multicollinearity
high_vif = vif_data[vif_data["VIF"] > 10]
if len(high_vif) > 0:
    print(
        f"\nWarning: High multicollinearity detected for: {high_vif['Feature'].tolist()}"
    )
else:
    print("\nNo significant multicollinearity detected (all VIF < 10)")

Records for RD prediction analysis: 90
Features used: ['OBP', 'OOBP', 'SLG', 'OSLG', 'BA']

=== Multicollinearity Check (VIF) ===
  Feature          VIF
0     OBP  1733.865062
1    OOBP  1183.902401
2     SLG   862.722495
3    OSLG   861.375910
4      BA  2251.406140



In [9]:
# Build multiple regression model using statsmodels for detailed statistics
# Add constant term for intercept
X_rd_with_const = sm.add_constant(X_rd)

# Fit the model
rd_model = sm.OLS(y_rd, X_rd_with_const).fit()

# Display detailed results
print("=== Multiple Regression: RD ~ OBP + OOBP + SLG + OSLG + BA ===")
print(rd_model.summary())

# Extract coefficients and p-values for analysis
coefficients = rd_model.params[1:]  # Exclude constant
p_values = rd_model.pvalues[1:]  # Exclude constant
conf_int = rd_model.conf_int()[1:]  # Exclude constant

# Create coefficient analysis dataframe
coef_analysis = pd.DataFrame(
    {
        "Feature": coefficients.index,
        "Coefficient": coefficients.values,
        "P-value": p_values.values,
        "Significant": p_values.values < 0.05,
        "Lower_CI": conf_int[0].values,
        "Upper_CI": conf_int[1].values,
    }
)

# Sort by absolute coefficient value (importance)
coef_analysis["Abs_Coefficient"] = abs(coef_analysis["Coefficient"])
coef_analysis = coef_analysis.sort_values("Abs_Coefficient", ascending=False)

print("\n=== Feature Importance Analysis ===")
print(
    coef_analysis[["Feature", "Coefficient", "P-value", "Significant"]].to_string(
        index=False
    )
)

# Identify the most important predictor
most_important = coef_analysis.iloc[0]
print(f"\n=== Key Finding ===")
print(f"Most important predictor: {most_important['Feature']}")
print(f"Coefficient: {most_important['Coefficient']:.2f}")
print(f"P-value: {most_important['P-value']:.4f}")
print(f"Significant: {most_important['Significant']}")

=== Multiple Regression: RD ~ OBP + OOBP + SLG + OSLG + BA ===
                            OLS Regression Results                            
Dep. Variable:                     RD   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                     173.1
Date:                Sat, 25 Oct 2025   Prob (F-statistic):           1.10e-42
Time:                        01:29:29   Log-Likelihood:                -443.75
No. Observations:                  90   AIC:                             899.5
Df Residuals:                      84   BIC:                             914.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [10]:
# Model performance for plot 4. Calculate R².
y_pred_rd = rd_model.predict(X_rd_with_const)
r2_rd = r2_score(y_rd, y_pred_rd)

# Plotly version: Run Difference (RD) Prediction Analysis (Pre-Moneyball Era)
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Feature Coefficients (Red = Significant, p<0.05)",
        "Statistical Significance (-log10 (p-value)",
        f'OBP vs RD<br>Correlation: {rd_data["OBP"].corr(rd_data["RD"]):.3f}, R² = {round(rd_data["OBP"].corr(rd_data["RD"])**2, 3)}',
        f"Model Performance<br>R² = {r2_rd:.3f}",
    ),
)

# Plot 1: Coefficient plot (sorted by importance)
features = coef_analysis["Feature"]
coeffs = coef_analysis["Coefficient"]
colors = ["red" if p < 0.05 else "gray" for p in coef_analysis["P-value"]]

fig.add_trace(
    go.Bar(
        y=features, x=coeffs, orientation="h", marker_color=colors, name="Coefficients"
    ),
    row=1,
    col=1,
)

# Add zero line
fig.add_vline(x=0, line_dash="dash", line_color="black", opacity=0.5, row=1, col=1)

# Add value labels
for i, (feature, coeff) in enumerate(zip(features, coeffs)):
    fig.add_annotation(
        x=coeff + (0.1 if coeff >= 0 else -0.1),
        y=feature,
        text=f"{coeff:.1f}",
        showarrow=False,
        font=dict(size=10, color="black"),
        row=1,
        col=1,
    )

# Plot 2: P-values (significance)
log_p_values = -np.log10(coef_analysis["P-value"])
log_p_values = np.minimum(log_p_values, 20)  # Cap at 20 for visualization

fig.add_trace(
    go.Bar(
        y=features,
        x=log_p_values,
        orientation="h",
        marker_color=colors,
        name="-log10(p-value)",
    ),
    row=1,
    col=2,
)

# Add p=0.05 threshold line
fig.add_vline(
    x=-np.log10(0.05), line_dash="dash", line_color="red", opacity=0.7, row=1, col=2
)

# Add note about -log10(0.05) = 1.30
fig.add_annotation(
    x=1.60,
    y=4,
    xref="paper",
    yref="paper",
    text="Note: -log10(0.05) = 1.30",
    showarrow=False,
    font=dict(size=10),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="gray",
    borderwidth=1,
    row=1,
    col=2,
)

fig.add_annotation(
    x=1.30,
    y=4.3,
    xref="paper",
    yref="paper",
    text="p = 0.05",
    showarrow=False,
    font=dict(size=10),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="gray",
    borderwidth=1,
    row=1,
    col=2,
)


# Add value labels for significant bars
for i, (feature, log_p) in enumerate(zip(features, log_p_values)):
    if log_p > 0.5:  # Only add labels if bar is tall enough
        fig.add_annotation(
            x=log_p + 0.1,
            y=feature,
            text=f"{log_p:.1f}",
            showarrow=False,
            font=dict(size=9, color="black"),
            row=1,
            col=2,
        )

# Plot 3: OBP vs RD scatter plot (most important feature)
fig.add_trace(
    go.Scatter(
        x=rd_data["OBP"],
        y=rd_data["RD"],
        mode="markers",
        marker=dict(color="blue", opacity=0.6),
        name="OBP vs RD",
    ),
    row=2,
    col=1,
)

# Add regression line
z = np.polyfit(rd_data["OBP"], rd_data["RD"], 1)
p = np.poly1d(z)
fig.add_trace(
    go.Scatter(
        x=rd_data["OBP"],
        y=p(rd_data["OBP"]),
        mode="lines",
        line=dict(color="red", dash="dash", width=2),
        name="Regression",
    ),
    row=2,
    col=1,
)

# Plot 4: Model performance
fig.add_trace(
    go.Scatter(
        x=y_rd,
        y=y_pred_rd,
        mode="markers",
        marker=dict(color="green", opacity=0.6),
        name="Actual vs Predicted",
    ),
    row=2,
    col=2,
)

# Add perfect prediction line
fig.add_trace(
    go.Scatter(
        x=[y_rd.min(), y_rd.max()],
        y=[y_rd.min(), y_rd.max()],
        mode="lines",
        line=dict(color="red", dash="dash", width=2),
        name="Perfect Prediction",
    ),
    row=2,
    col=2,
)

# Update axes labels
fig.update_xaxes(title_text="Coefficient Value", row=1, col=1)
fig.update_yaxes(title_text="Features", row=1, col=1)
fig.update_xaxes(title_text="-log10 (p-value)", row=1, col=2)
fig.update_yaxes(title_text="Features", row=1, col=2)
fig.update_xaxes(title_text="On-Base Percentage (OBP)", row=2, col=1)
fig.update_yaxes(title_text="Run Difference (RD)", row=2, col=1)
fig.update_xaxes(title_text="Actual RD", row=2, col=2)
fig.update_yaxes(title_text="Predicted RD", row=2, col=2)

fig.update_layout(
    title_text="Run Difference (RD) Prediction Analysis (Pre-Moneyball Era)",
    title_x=0.5,
    width=1400,
    height=900,
    showlegend=False,
    margin=dict(l=60, r=40, t=100, b=60),
)

fig.show()

## 4. Analysis Part 3: Find And Hire Undervalued Players

**Question:** Which players are considered as undervalued?

**Goal:** Find undervalued players.


In [11]:
# Load the enhanced moneyball dataset
enhanced_df = pd.read_csv("enhanced_moneyball_analysis_1990_2010.csv")

# Print basic dataset information
print("=== ENHANCED MONEYBALL DATASET ANALYSIS ===")
print(f"Total number of records: {enhanced_df.shape[0]:,}")
print(f"Year range: {enhanced_df['yearID'].min()} - {enhanced_df['yearID'].max()}")
print(f"Number of unique players: {enhanced_df['playerID'].nunique():,}")

# Filter for pre-Moneyball era (yearID <= 2001) and (salary > 0)
pre_moneyball_df = enhanced_df[
    (enhanced_df["yearID"] <= 2001) & (enhanced_df["salary"] > 0)
].copy()

# pre_moneyball_df = pre_moneyball_df[pre_moneyball_df.AB >= 55]

print(f"\n=== PRE-MONEYBALL ERA (≤2001) ANALYSIS ===")
print(f"Total number of records: {pre_moneyball_df.shape[0]:,}")
print(
    f"Year range: {pre_moneyball_df['yearID'].min()} - {pre_moneyball_df['yearID'].max()}"
)
print(f"Number of unique players: {pre_moneyball_df['playerID'].nunique():,}")

pre_moneyball_df.head()

=== ENHANCED MONEYBALL DATASET ANALYSIS ===
Total number of records: 5,040
Year range: 1990 - 2010
Number of unique players: 1,046

=== PRE-MONEYBALL ERA (≤2001) ANALYSIS ===
Total number of records: 2,800
Year range: 1990 - 2001
Number of unique players: 677


,playerID,yearID,teamID,G,AB,H,BB,HR,BA,OBP,SLG,OPS,salary,Salary_per_OBP,Salary_per_OPS,OBP_Value_Score,OPS_Value_Score
0,abbotku01,1994,FLO,101,345,86,16,9,0.249275,0.290761,0.394203,0.684964,109000,3.748785e+05,1.591325e+05,12.228428,15.501188
1,abbotku01,1995,FLO,120,420,107,36,17,0.254762,0.317597,0.452381,0.769978,119000,3.746892e+05,1.545500e+05,17.139004,21.887725
2,abbotku01,1996,FLO,109,320,81,22,8,0.253125,0.307246,0.428125,0.735371,250000,8.136792e+05,3.399643e+05,11.720460,14.822770
3,abbotku01,1999,COL,96,286,78,16,8,0.272727,0.310231,0.430070,0.740301,900000,2.901064e+06,1.215722e+06,5.737146,7.132091
5,abreubo01,1998,PHI,151,497,155,84,17,0.311871,0.408547,0.496982,0.905529,180000,4.405858e+05,1.987789e+05,30.553799,28.719534


In [12]:
# Remove rows with missing OBP or salary data for the analysis
analysis_df = pre_moneyball_df.dropna(subset=["OBP", "salary"]).copy()

# Calculate percentiles to identify undervalued players
# High OBP = above 75th percentile
# Low Salary = below 25th percentile
obp_75th = analysis_df["OBP"].quantile(0.75)
salary_25th = analysis_df["salary"].quantile(0.25)

print(f"=== QUADRANT ANALYSIS CRITERIA ===")
print(f"High OBP threshold (75th percentile): {obp_75th:.3f}")
print(f"Low Salary threshold (25th percentile): ${salary_25th:,.0f}")


# Create quadrant categories for each player
def assign_quadrant(row):
    if row["OBP"] >= obp_75th and row["salary"] <= salary_25th:
        return "High OBP, Low Salary (Green)"  # Undervalued
    elif row["OBP"] < obp_75th and row["salary"] <= salary_25th:
        return "Low OBP, Low Salary (Blue)"
    elif row["OBP"] >= obp_75th and row["salary"] > salary_25th:
        return "High OBP, High Salary (Gray)"
    else:
        return "Low OBP, High Salary (Red)"  # Overvalued


analysis_df["quadrant"] = analysis_df.apply(assign_quadrant, axis=1)

# Count players in each quadrant
quadrant_counts = analysis_df["quadrant"].value_counts()
print(f"\n=== QUADRANT DISTRIBUTION ===")
for quadrant, count in quadrant_counts.items():
    print(f"{quadrant}: {count} players")

# Identify undervalued players (green quadrant)
undervalued_players = analysis_df[
    analysis_df["quadrant"] == "High OBP, Low Salary (Green)"
].copy()
print(f"\nNumber of undervalued players identified: {len(undervalued_players)}")

# Create interactive scatter plot with quadrant-based coloring
fig = px.scatter(
    analysis_df,
    x="salary",
    y="OBP",
    color="quadrant",  # Color by quadrant
    size="G",  # Games played as size
    size_max=8,  # Smaller maximum bubble size
    hover_data=["playerID", "teamID", "yearID", "BA", "SLG", "OPS"],
    title="Quadrant Analysis: OBP vs Salary (Pre-Moneyball Era)",
    labels={
        "salary": "Salary ($)",
        "OBP": "On-Base Percentage (OBP)",
        "G": "Games Played",
        "quadrant": "Quadrant",
    },
    color_discrete_map={
        "High OBP, Low Salary (Green)": "green",
        "Low OBP, Low Salary (Blue)": "blue",
        "High OBP, High Salary (Gray)": "gray",
        "Low OBP, High Salary (Red)": "red",
    },
)

# Add threshold lines
fig.add_hline(
    y=obp_75th,
    line_dash="dash",
    line_color="red",
    annotation_text=f"High OBP Threshold ({obp_75th:.3f})",
)
fig.add_vline(
    x=salary_25th,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Low Salary Threshold (${salary_25th:,.0f})",
)

# Quadrant-based coloring is now handled by the main scatter plot

# Update layout
fig.update_layout(
    width=1200,
    height=600,
    showlegend=False,
    xaxis_title="Salary ($)",
    yaxis_title="On-Base Percentage (OBP)",
    title_x=0.5,
)

# Add quadrant labels
fig.add_annotation(
    x=0.95,
    y=0.95,
    xref="paper",
    yref="paper",
    text="<b>High OBP<br>High Salary</b>",
    showarrow=False,
    font=dict(size=12, color="gray"),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="gray",
    borderwidth=1,
)

fig.add_annotation(
    x=0.05,
    y=0.95,
    xref="paper",
    yref="paper",
    text="<b>High OBP<br>Low Salary</b><br><span style='color:green'>UNDERVALUED</span>",
    showarrow=False,
    font=dict(size=12, color="green"),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="green",
    borderwidth=2,
)

fig.add_annotation(
    x=0.95,
    y=0.05,
    xref="paper",
    yref="paper",
    text="<b>Low OBP<br>High Salary</b><br><span style='color:red'>OVERVALUED</span>",
    showarrow=False,
    font=dict(size=12, color="red"),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="red",
    borderwidth=1,
)

fig.add_annotation(
    x=0.05,
    y=0.05,
    xref="paper",
    yref="paper",
    text="<b>Low OBP<br>Low Salary</b>",
    showarrow=False,
    font=dict(size=12, color="blue"),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="blue",
    borderwidth=1,
)

fig.show()

# Display top undervalued players if any exist
if len(undervalued_players) > 0:
    print(f"\n=== UNDERVALUED PLAYERS (High OBP (descending), Low Salary) ===\n")
    top_undervalued = undervalued_players.sort_values(["OBP"], ascending=[False])[
        ["playerID", "yearID", "teamID", "OBP", "BA", "SLG", "OPS", "salary"]
    ]
    print(top_undervalued.to_string(index=False))
else:
    print("\nNo players meet the strict criteria for undervalued players.")
    print("Let's look at players with above-average OBP and below-average salary...")

    # Let us relax the criteria to find more undervalued players.
    # Relaxed criteria:
    # 1. Above median OBP
    # 2. Below median salary
    obp_median = analysis_df["OBP"].median()
    salary_median = analysis_df["salary"].median()

    relaxed_undervalued = analysis_df[
        (analysis_df["OBP"] >= obp_median) & (analysis_df["salary"] <= salary_median)
    ].copy()

    print(
        f"\nRelaxed criteria (above median OBP: {obp_median:.3f}, below median salary: ${salary_median:,.0f})"
    )
    print(f"Number of players: {len(relaxed_undervalued)}")

    if len(relaxed_undervalued) > 0:
        top_relaxed = relaxed_undervalued.sort_values(["OBP"], ascending=[False])[
            ["playerID", "yearID", "teamID", "OBP", "BA", "SLG", "OPS", "salary"]
        ]
        print(top_relaxed.to_string(index=False))

=== QUADRANT ANALYSIS CRITERIA ===
High OBP threshold (75th percentile): 0.371
Low Salary threshold (25th percentile): $304,375

=== QUADRANT DISTRIBUTION ===
Low OBP, High Salary (Red): 1481 players
Low OBP, Low Salary (Blue): 619 players
High OBP, High Salary (Gray): 619 players
High OBP, Low Salary (Green): 81 players

Number of undervalued players identified: 81



=== UNDERVALUED PLAYERS (High OBP (descending), Low Salary) ===

 playerID  yearID teamID      OBP       BA      SLG      OPS  salary
thomafr04    1991    CHA 0.452857 0.318426 0.552773 1.005630  120000
magadda01    1997    OAK 0.413580 0.302583 0.391144 0.804724  255000
shumpte01    1999    COL 0.413333 0.347328 0.583969 0.997303  220000
greerru01    1994    TEX 0.410334 0.314079 0.487365 0.897699  109000
leyriji01    1993    NYA 0.409836 0.308880 0.525097 0.934933  152000
abreubo01    1998    PHI 0.408547 0.311871 0.496982 0.905529  180000
millira01    1990    BAL 0.407895 0.265193 0.491713 0.899607  155000
higgibo02    1996    DET 0.404297 0.320455 0.577273 0.981570  170000
ochoaal01    1999    MIL 0.404255 0.299639 0.465704 0.869959  245000
glaustr01    2000    ANA 0.404130 0.284192 0.603908 1.008037  275000
younger01    1995    COL 0.403800 0.316940 0.472678 0.876478  252500
tatisfe01    1999    SLN 0.403756 0.297952 0.553073 0.956828  270000
pujolal01    2001    SLN 0.402963 0.3

In [13]:
# Players mentioned in the vide below and found in the list above.
# Video: https://www.youtube.com/watch?v=3MjxoaynCmk
# Jeremy Giambi - Player ID: giambje01
# David Justice - Player ID: justida01
# Scott Hatteberg - Player ID: hattesc01